In [42]:
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV,RandomizedSearchCV
from sklearn.feature_selection import RFECV,SelectFromModel
from sklearn.metrics import classification_report
from xgboost import XGBClassifier
from sklearn.feature_selection import VarianceThreshold
import numpy as np
from sklearn.metrics import roc_auc_score
from xgboost import XGBClassifier
from sklearn.model_selection import ParameterSampler
from scipy.stats import uniform, randint
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import ParameterSampler
from scipy.stats import randint, uniform
import numpy as np


In [43]:
import pandas as pd
train=pd.read_csv('../artifacts/training_data.csv',low_memory=True, dtype=np.float32)
test=pd.read_csv('../artifacts/testing_data.csv',low_memory=True, dtype=np.float32)



In [44]:
dep = ['Label']
X_train = train.loc[:,~train.columns.isin(dep)]
y_train = train.loc[:,train.columns.isin(dep)]

In [45]:
X_val = test.loc[:,~test.columns.isin(dep)]
y_val = test.loc[:,test.columns.isin(dep)]

In [46]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import ParameterSampler
from scipy.stats import randint, uniform
import numpy as np

# Step 1: Fit base model on all features
base_model = RandomForestClassifier(n_estimators=100, random_state=42)
base_model.fit(X_train, y_train)

# Step 2: Select important features only
selector = SelectFromModel(base_model, threshold="median", prefit=True)
X_train_sel = selector.transform(X_train)
X_val_sel = selector.transform(X_val)


c:\Users\ASUS\Desktop\Assignment - Sigmoid\model-dev\Lib\site-packages\sklearn\base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\ASUS\Desktop\Assignment - Sigmoid\model-dev\Lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
c:\Users\ASUS\Desktop\Assignment - Sigmoid\model-dev\Lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


In [47]:
y_train.mean()

Label    0.073314
dtype: float32

In [48]:
# Define param grid
# Step 3: Define hyperparameter search space
param_dist = { 
    'n_estimators': randint(100, 500),
    'max_depth': randint(2, 8),
    'min_samples_split': randint(2, 10),
    'min_samples_leaf': randint(1, 5),
    'max_features': ['sqrt', 'log2', None],
    'bootstrap': [True, False]
}

n_iter = 5
param_list = list(ParameterSampler(param_dist, n_iter=n_iter, random_state=42))

# Step 4: Hyperparameter tuning loop
best_score = -1
best_model = None
best_params = None
results = []

for i, params in enumerate(param_list):
    print(params)
    model = RandomForestClassifier(random_state=42, **params, n_jobs=-1)
    model.fit(X_train_sel, y_train.values.ravel())

    # Predict probabilities and compute AUC
    y_val_prob = model.predict_proba(X_val_sel)[:, 1]
    val_auc = roc_auc_score(y_val, y_val_prob)
    print(val_auc)
    results.append({'params': params, 'val_auc': val_auc})

    if val_auc > best_score:
        best_score = val_auc
        best_model = model
        best_params = params

{'bootstrap': True, 'max_depth': 5, 'max_features': 'sqrt', 'min_samples_leaf': 3, 'min_samples_split': 4, 'n_estimators': 171}
0.5938352733210454
{'bootstrap': True, 'max_depth': 6, 'max_features': None, 'min_samples_leaf': 2, 'min_samples_split': 4, 'n_estimators': 314}
0.5880664595356575
{'bootstrap': True, 'max_depth': 4, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 9, 'n_estimators': 251}
0.5913715795986716
{'bootstrap': True, 'max_depth': 7, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 9, 'n_estimators': 393}
0.5938474749791173
{'bootstrap': False, 'max_depth': 5, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 413}
0.5928263426183937


In [49]:
results

[{'params': {'bootstrap': True,
   'max_depth': 5,
   'max_features': 'sqrt',
   'min_samples_leaf': 3,
   'min_samples_split': 4,
   'n_estimators': 171},
  'val_auc': np.float64(0.5938352733210454)},
 {'params': {'bootstrap': True,
   'max_depth': 6,
   'max_features': None,
   'min_samples_leaf': 2,
   'min_samples_split': 4,
   'n_estimators': 314},
  'val_auc': np.float64(0.5880664595356575)},
 {'params': {'bootstrap': True,
   'max_depth': 4,
   'max_features': 'sqrt',
   'min_samples_leaf': 4,
   'min_samples_split': 9,
   'n_estimators': 251},
  'val_auc': np.float64(0.5913715795986716)},
 {'params': {'bootstrap': True,
   'max_depth': 7,
   'max_features': 'sqrt',
   'min_samples_leaf': 2,
   'min_samples_split': 9,
   'n_estimators': 393},
  'val_auc': np.float64(0.5938474749791173)},
 {'params': {'bootstrap': False,
   'max_depth': 5,
   'max_features': 'sqrt',
   'min_samples_leaf': 1,
   'min_samples_split': 5,
   'n_estimators': 413},
  'val_auc': np.float64(0.59282634261

In [50]:
best_model

RandomForestClassifier(max_depth=7, min_samples_leaf=2, min_samples_split=9,
                       n_estimators=393, n_jobs=-1, random_state=42)

In [51]:
best_score

np.float64(0.5938474749791173)

In [52]:
#best_model.save_model("../model-objects/RF_final_model.json")  # Saves model to JSON format

import pickle

with open('../model-objects/RF_final_model.pkl', 'wb') as f:
    pickle.dump(best_model, f)

# #loaded_model = XGBClassifier()
# #loaded_model.load_model("../model-objects/xgb_final_model.json")


In [53]:
# Step 6: Get predictions on train/val
y_train_prob = best_model.predict_proba(X_train_sel)[:, 1]
y_val_prob = best_model.predict_proba(X_val_sel)[:, 1]

In [54]:
data_ap_train = pd.DataFrame({'Actual':y_train.values.ravel(),'Predicted':y_train_prob})
data_ap_val = pd.DataFrame({'Actual':y_val.values.ravel(),'Predicted':y_val_prob})

**KS AND RANK ORDER**

In [55]:
def ks(data=None,target=None, prob=None):
    data['target0'] = 1 - data[target]
    data['bucket'] = pd.qcut(data[prob], 10)
    grouped = data.groupby('bucket', as_index = False)
    kstable = pd.DataFrame()
    kstable['min_prob'] = grouped.min()[prob]
    kstable['max_prob'] = grouped.max()[prob]
    kstable['events']   = grouped.sum()[target]
    kstable['nonevents'] = grouped.sum()['target0']
    kstable = kstable.sort_values(by="min_prob", ascending=False).reset_index(drop = True)
    kstable['event_rate'] = (kstable.events / data[target].sum()).apply('{0:.2%}'.format)
    kstable['nonevent_rate'] = (kstable.nonevents / data['target0'].sum()).apply('{0:.2%}'.format)
    kstable['cum_eventrate']=(kstable.events / data[target].sum()).cumsum()
    kstable['cum_noneventrate']=(kstable.nonevents / data['target0'].sum()).cumsum()
    kstable['KS'] = np.round(kstable['cum_eventrate']-kstable['cum_noneventrate'], 3) * 100

    #Formating
    kstable['cum_eventrate']= kstable['cum_eventrate'].apply('{0:.2%}'.format)
    kstable['cum_noneventrate']= kstable['cum_noneventrate'].apply('{0:.2%}'.format)
    kstable.index = range(1,11)
    kstable.index.rename('Decile', inplace=True)
    pd.set_option('display.max_columns', 9)
    print(kstable)
    
    #Display KS
    from colorama import Fore
    print(Fore.RED + "KS is " + str(max(kstable['KS']))+"%"+ " at decile " + str((kstable.index[kstable['KS']==max(kstable['KS'])][0])))
    return(kstable)

In [56]:
ks_train=ks(data=data_ap_train,target='Actual',prob='Predicted')

        min_prob  max_prob  events  nonevents event_rate nonevent_rate  \
Decile                                                                   
1       0.135541  0.749524  1720.0     1780.0     67.03%         5.49%   
2       0.081024  0.135484   355.0     3145.0     13.83%         9.70%   
3       0.061530  0.081019   169.0     3331.0      6.59%        10.27%   
4       0.051263  0.061526   101.0     3399.0      3.94%        10.48%   
5       0.045740  0.051259    60.0     3440.0      2.34%        10.61%   
6       0.042678  0.045738    56.0     3444.0      2.18%        10.62%   
7       0.040988  0.042678    49.0     3451.0      1.91%        10.64%   
8       0.039817  0.040986    27.0     3473.0      1.05%        10.71%   
9       0.039058  0.039817    25.0     3474.0      0.97%        10.71%   
10      0.038069  0.039058     4.0     3497.0      0.16%        10.78%   

       cum_eventrate cum_noneventrate         KS  
Decile                                            
1        

C:\Users\ASUS\AppData\Local\Temp\ipykernel_14772\572846000.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = data.groupby('bucket', as_index = False)


In [57]:
ks_val=ks(data=data_ap_val,target='Actual',prob='Predicted')

        min_prob  max_prob  events  nonevents event_rate nonevent_rate  \
Decile                                                                   
1       0.125430  0.587928   192.0     1308.0     17.36%         9.41%   
2       0.075308  0.125425   150.0     1350.0     13.56%         9.72%   
3       0.057291  0.075301   132.0     1368.0     11.93%         9.85%   
4       0.048589  0.057289   111.0     1389.0     10.04%        10.00%   
5       0.044029  0.048583   110.0     1390.0      9.95%        10.00%   
6       0.041693  0.044028    86.0     1414.0      7.78%        10.18%   
7       0.040379  0.041692    82.0     1418.0      7.41%        10.21%   
8       0.039519  0.040379    95.0     1405.0      8.59%        10.11%   
9       0.038933  0.039519    78.0     1422.0      7.05%        10.23%   
10      0.038078  0.038932    70.0     1430.0      6.33%        10.29%   

       cum_eventrate cum_noneventrate         KS  
Decile                                            
1        

C:\Users\ASUS\AppData\Local\Temp\ipykernel_14772\572846000.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = data.groupby('bucket', as_index = False)


In [58]:
ks_train.to_csv('../data-analysis/ks_table_train_rf.csv')
ks_val.to_csv('../data-analysis/ks_table_val_rf.csv')